 ### simple function : 

In [1]:
def outer():
    # this is outer  enclosing function
    message = 'Hi'
     # this is nested function 
    def inner():
        print(message)
    return inner()

*A function defined inside another function is called a nested function. Nested functions can access variables of the enclosing scope.

In [2]:
outer()

Hi


instead on running inner() function, we have returned the function as return inner

### Closures in python

* A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory

* A closure lets us invoke Python function outside its scope.



In [3]:
def outer(msg):
    # This is the outer enclosing function
    message =msg

    def inner():
        # This is the nested function
        print(message)

    return inner  # Note we are returning function WITHOUT parenthesis 

hello_func = outer("Hello")
bye_func = outer('bye')
hello_func

<function __main__.outer.<locals>.inner()>

In [4]:
print(hello_func.__name__)

inner


 * here hello_func() and bye_func() acts as a function and return a inner function

In [5]:
hello_func()
bye_func()

Hello
bye


In [6]:
print(type(hello_func))
print(hello_func.__closure__)
print(hello_func.__closure__[0].cell_contents)

<class 'function'>
(<cell at 0x000001E37BE075B8: str object at 0x000001E37BDEAC70>,)
Hello


##### * we can see each cell of the objects retains the value at the time of its creation.

In [7]:
print(type(bye_func()))
print(bye_func.__closure__)
print(bye_func.__closure__[0].cell_contents)

bye
<class 'NoneType'>
(<cell at 0x000001E37BE074F8: str object at 0x000001E37BDEAEF0>,)
bye


* The outer() function was called with the string "Hello" and 'bye'and the returned function was bound to the name hello_func and bye_func respectively

* On calling hello_func() and bye_func(), the message was still remembered although we had already finished executing the print_msg() function.

* This technique by which some data ("Hello in this case) gets attached to the code is called closure in Python.

In [8]:
del outer

In [9]:
hello_func()

Hello


##### * This value in the enclosing scope is remembered even when the variable goes out of scope or the function itself is removed from the current namespace.Here, the returned function still works even when the original function was deleted.



In [10]:
outer('Hello')

NameError: name 'outer' is not defined

## Decorators

###### * A decorator is a function that takes another function and extends the behavior of this function without explicitly modifying it

* A function is decorated with the @ symbol:

In [11]:
#@my_decorator
def my_function():
    pass

#### EX:1

In [12]:
def decorator_func(original_func):
    def wrapper_func():
        print(f'wrapper executed this before {original_func.__name__}')
        return original_func()
    return wrapper_func

def display():
    print(f'display func ran')

* Python are first class objects, which means that – like any other object – they can be defined inside another function, passed as argument to another function, or returned from other functions.

##### * A decorator is a function that takes another function as argument, wraps its behaviour inside an inner function. and returns the wrapped function. As a consequence, the decorated function no has extended functionality!

In [13]:
decorator_display = decorator_func(display)
decorator_display()

wrapper executed this before display
display func ran


* Instead of wrapping our function and asigning it to itself, we can achieve the same thing simply by decorating our function with an @.

In [14]:
# added decorator 
def decorator_func(original_func):
    def wrapper_func():
        print(f'wrapper executed this before {original_func.__name__}')
        return original_func()
    return wrapper_func

@decorator_func  # this function needs to be decorated
def display():
    print(f'display func ran')
    
display()

wrapper executed this before display
display func ran


##### EX: 2

In [15]:
# A decorator function takes another function as argument, wraps its behaviour inside
# an inner function, and returns the wrapped function.
def start_end_decorator(func):
    
    def wrapper():
        print('Start')
        func()
        print('End')
    return wrapper

def print_name():
    print('Alex')


In [16]:
print_name()

Alex


In [17]:
# Now wrap the function by passing it as argument to the decorator function
# and asign it to itself -> Our function has extended behaviour!
print_name = start_end_decorator(print_name)
print_name()

Start
Alex
End


In [18]:
# using decorator syntax
@start_end_decorator
def print_name():
    print('Alex')
    
print_name()

Start
Alex
End


#### EX: 3 Decorator function arguments

In [19]:
def evening_greeting(func):
    def function_wrapper(x):
        print("Good evening, " + func.__name__ + " returns:")
        func(x)
    return function_wrapper

@evening_greeting
def foo(x):
    print(42)
foo('Hi')

Good evening, foo returns:
42


In [22]:
import functools
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")
    
greet('Alex')

Hello Alex
Hello Alex
Hello Alex


#### EX: 4

In [23]:
def start_end_decorator_2(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_2  # now add_5 has new functinality , which print start , end  and number
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
15


In [24]:
print(add_5.__name__)
help(add_5)

wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



If we have a look at the name of our decorated function, and inspect it with the built-in help function, we notice that Python thinks our function is now the wrapped inner function of the decorator function.


* To fix this, 
##### use the functools.wraps decorator, 
which will preserve the information about the original function. This is helpful for introspection purposes, i.e. the ability of an object to know about its own attributes at runtime:


In [25]:
import functools
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_4
def add_5(x):
    return x + 5
result = add_5(10)
print(result)
print(add_5.__name__)
help(add_5)

Start
End
15
add_5
Help on function add_5 in module __main__:

add_5(x)



![img.png](https://miro.medium.com/proxy/1*1815O8Ytu4y0Rx6l0-v--A.png)

#### EX: 5 why we need wrapper function

* without wrapper

In [26]:
import time
# function to calculate the square ,added time functionality to compute execution time
def calc_square(numbers):
    start = time.time()
    result = []
    for number in numbers:
        result.append(number*number)
    end = time.time()
    print(calc_square.__name__ +" took " + str((end-start)*1000) + "mil sec")
    return result

# function to calculate cube ,added time functionality to compute execution time
def calc_cube(numbers):
    start = time.time()
    result = []
    for number in numbers:
        result.append(number*number*number)
    end = time.time()
    print(calc_cube.__name__ +" took " + str((end-start)*1000) + "mil sec")
    return result

In [27]:
array = range(1,100000)
out_square = calc_square(array)
out_cube = calc_cube(array)

calc_square took 43.90692710876465mil sec
calc_cube took 94.3455696105957mil sec


###### * with decorator . we have decorated cal_sqaure and cal_cube function with time functionality time_it. 

In [28]:
import time
def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        end = time.time()
        print(func.__name__ +" took " + str((end-start)*1000) + "mil sec")
        return result
    return wrapper

In [29]:
@time_it
def calc_square(numbers):
    result = []
    for number in numbers:
        result.append(number*number)
    return result

@time_it
def calc_cube(numbers):
    result = []
    for number in numbers:
        result.append(number*number*number)
    return result

array = range(1,100000)
out_square = calc_square(array)
out_cube = calc_cube(array)

calc_square took 46.26035690307617mil sec
calc_cube took 48.36082458496094mil sec


##### Nested Decorators, We can apply several decorators to a function by stacking them on top of each other. The decorators are being executed in the order they are listed.

#### Ex: 6 . class decorator 

In [30]:
#without class implementation
def decorator_func(original_func):
    def wrapper_func(*args,**kwargs):
        print(f'wrapped executed before {original_func.__name__}')
        return original_func(*args,**kwargs)
    return wrapper_func
@decorator_func
def display():
    print('display function ran')
@decorator_func
def display_info(name, age):
    print(f'display_info ran with arguments {name},{age}')
    
display()
display_info('john',25)

wrapped executed before display
display function ran
wrapped executed before display_info
display_info ran with arguments john,25


######  * with class implementation

* We can also use a class as a decorator. Therefore, we have to implement the __call__() method to make our object callable.

In [31]:
class decorator_class(object):
    def __init__(self, original_func):
        self.original_func = original_func
    def __call__(self, *args,**kwargs):
        print(f'call method executed before {self.original_func.__name__}')
        return self.original_func(*args,**kwargs)
@decorator_class
def display():
    print('display function ran')
@decorator_class
def display_info(name, age):
    print(f'display_info ran with arguments {name},{age}')
    
display()
display_info('john',25)       

call method executed before display
display function ran
call method executed before display_info
display_info ran with arguments john,25
